### Initialisation

In [1]:
!pip install flair
!pip install sentence-transformers
!pip install yellowbrick
!pip install  wordcloud

     |████████████████████████████████| 286kB 5.0MB/s eta 0:00:01
     |████████████████████████████████| 776.8MB 20kB/s 
     |████████████████████████████████| 798kB 51.8MB/s 
     |████████████████████████████████| 1.2MB 51.3MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 2.3MB 48.3MB/s 
     |████████████████████████████████| 19.7MB 541kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 983kB 45.8MB/s 
     |████████████████████████████████| 3.3MB 54.2MB/s 
     |████████████████████████████████| 901kB 54.4MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=1f33ad4acb72c572f01ec028be73f92618dd626b3ab4830fd3f18ed1d132a0ab
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for mpld3

In [2]:
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score, train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, auc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import time
from scipy.sparse import csr_matrix, hstack
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
from flair.embeddings import FlairEmbeddings, TransformerWordEmbeddings , TransformerDocumentEmbeddings, SentenceTransformerDocumentEmbeddings
from flair.data import Sentence
from sklearn.model_selection import learning_curve
from sklearn.linear_model import *
from sklearn.preprocessing import LabelEncoder
from yellowbrick.model_selection import LearningCurve
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import *
import io
from google.colab import files
import pickle
from wordcloud import WordCloud

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
uploaded = files.upload()

Saving train_tweets.pkl to train_tweets.pkl
Saving test_tweets.pkl to test_tweets.pkl


In [7]:
df_train = pd.read_pickle(io.BytesIO(uploaded['train_tweets.pkl']))
df_test = pd.read_pickle(io.BytesIO(uploaded['test_tweets.pkl']))

In [8]:
argumentative_tweets = pd.read_pickle('/content/drive/MyDrive/argumentative_tweets.pkl')

In [9]:
le = LabelEncoder()
def encoding(label):
        label_le= le.fit_transform(label)
        return label_le

df_train['Stance'] = encoding(df_train['Stance'])
df_test['Stance'] =  encoding(df_test['Stance'])


df_train_hillary = df_train.loc[ (df_train.Target == 'Hillary Clinton')]
df_train_abortion =  df_train.loc[ (df_train['Target'] == 'Legalization of Abortion')]
df_train_atheism =  df_train.loc[(df_train['Target'] == 'Atheism')]
df_train_climatechange = df_train.loc[(df_train['Target'] == 'Climate Change is a Real Concern') ] 
df_train_feministmovement = df_train.loc[(df_train['Target'] == 'Feminist Movement') ]

df_test_hillary = df_test.loc[(df_test['Target'] == 'Hillary Clinton') ]
df_test_abortion =  df_test.loc[ (df_test['Target'] == 'Legalization of Abortion') ]
df_test_atheism =  df_test.loc[(df_test['Target'] == 'Atheism') ]
df_test_climatechange = df_test.loc[(df_test['Target'] == 'Climate Change is a Real Concern')]
df_test_feministmovement = df_test.loc[(df_test['Target'] == 'Feminist Movement')]

In [10]:
# classifiers = []
# classifiers.append(RandomForestClassifier())
# classifiers.append(MLPClassifier())
# classifiers.append(SVC(kernel='rbf'))
# classifiers.append(LinearSVC(max_iter=2000))
# classifiers.append(LogisticRegression(solver='liblinear'))
classifiers = []
classifiers.append(LinearSVC(max_iter=1500))

### Train Test each target

In [11]:
folds = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)
to_join = lambda x : " ".join(x)
df_train_hillary['train'] = df_train_hillary.tokenize.apply(to_join)
df_test_hillary['test'] = df_test_hillary.tokenize.apply(to_join)
Train_X_hillary, Test_X_hillary, Train_Y_hillary, Test_Y_hillary = df_train_hillary['train'], df_test_hillary['test'], df_train_hillary['Stance'], df_test_hillary['Stance']

Train_Y_hillary = np.array(Train_Y_hillary)
Test_Y_hillary = np.array(Test_Y_hillary)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
df_train_atheism['train'] = df_train_atheism.tokenize.apply(to_join)
df_test_atheism['test'] = df_test_atheism.tokenize.apply(to_join)
Train_X_atheism, Test_X_atheism, Train_Y_atheism, Test_Y_atheism = df_train_atheism['train'], df_test_atheism['test'], df_train_atheism['Stance'], df_test_atheism['Stance']

Train_Y_atheism = np.array(Train_Y_atheism)
Test_Y_atheism = np.array(Test_Y_atheism)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
df_train_abortion['train'] = df_train_abortion.tokenize.apply(to_join)
df_test_abortion['test'] = df_test_abortion.tokenize.apply(to_join)
Train_X_abortion, Test_X_abortion, Train_Y_abortion, Test_Y_abortion = df_train_abortion['train'], df_test_abortion['test'], df_train_abortion['Stance'], df_test_abortion['Stance']

Train_Y_abortion = np.array(Train_Y_abortion)
Test_Y_abortion = np.array(Test_Y_abortion)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df_train_climatechange['train'] = df_train_climatechange.tokenize.apply(to_join)
df_test_climatechange['test'] = df_test_climatechange.tokenize.apply(to_join)
Train_X_climatechange, Test_X_climatechange, Train_Y_climatechange, Test_Y_climatechange = df_train_climatechange['train'], df_test_climatechange['test'], df_train_climatechange['Stance'], df_test_climatechange['Stance']

Train_Y_climatechange = np.array(Train_Y_climatechange)
Test_Y_climatechange = np.array(Test_Y_climatechange)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
df_train_feministmovement['train'] = df_train_feministmovement.tokenize.apply(to_join)
df_test_feministmovement['test'] = df_test_feministmovement.tokenize.apply(to_join)
Train_X_feministmovement, Test_X_feministmovement, Train_Y_feministmovement, Test_Y_feministmovement= df_train_feministmovement['train'], df_test_feministmovement['test'], df_train_feministmovement['Stance'], df_test_feministmovement['Stance']

Train_Y_feministmovement = np.array(Train_Y_feministmovement)
Test_Y_feministmovement = np.array(Test_Y_feministmovement)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:

df_train['train'] = df_train.tokenize.apply(to_join)
df_test['test'] = df_test.tokenize.apply(to_join)
Train_X, Test_X ,Train_Y, Test_Y = df_train['train'], df_test['test'], df_train['Stance'], df_test['Stance']

Train_Y = np.array(Train_Y)
Test_Y = np.array(Test_Y)

# Text Representation Vectors


In [17]:
bert_emb = SentenceTransformerDocumentEmbeddings('roberta-base')
model_w2v = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin', binary=True)
model_glove =pd.read_pickle('/content/drive/MyDrive/glove_dict.pickle')


Exception when trying to download http://sbert.net/models/roberta-base.zip. Response 404
SentenceTransformer-Model http://sbert.net/models/roberta-base.zip not found. Try to create it from scratch
Try to create Transformer Model roberta-base with mean pooling


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Run 

In [18]:
# function to get tfidf * word emb vector 
def tfidf_scaled_wv(data,model,dim):
    text = []
    for i in data['tokenize']:
        string = ' '.join(i)
        text.append(string)
    tfidf = TfidfVectorizer(stop_words =None)
    tf_idf_tr = tfidf.fit_transform(text)
    feature_names = tfidf.get_feature_names()
    sentences = data['tokenize'].values
    data_dtm = pd.DataFrame(tf_idf_tr.toarray(), columns = feature_names)
    tfidf_sent_vectors = [] 
    row = 0
    for sentence in sentences:
        sent_vec = np.zeros(dim)
        weight_sum = 0
        for word in sentence:
            try:
                vec = model[word]
                tfidf = data_dtm.iloc[row][word]
                sent_vec += (vec * tfidf)
                weight_sum += tfidf
            except:
                pass
            # check if weight_sum is zero meaning all words in that tweet gave no context 
        if weight_sum != 0:     
            sent_vec  /= weight_sum     
        tfidf_sent_vectors.append(sent_vec)
        row+=1
    return tfidf_sent_vectors
    

In [35]:
def get_embeddings(text):
    sentence = Sentence(text)
    bert_emb.embed(sentence)
    return sentence.embedding




def sentence_vec(data):
    # the returned embedding is a tensor so we used the pytorch method to detach into a numpy array
    detach = lambda x : x.detach().numpy()
    emb  = lambda x : get_embeddings(x)
    data['sen_emb'] = data.Processed_Tweet.apply(emb)
    data['sen_emb'] = data.sen_emb.apply(detach)
    # data['sen_emb'] = data.sen_emb.apply(lambda x: np.array(x))

In [20]:
def apply_embeddings(df_train,df_test,model,dim):
   train_emb = tfidf_scaled_wv(df_train,model,dim)
   test_emb = tfidf_scaled_wv(df_test,model,dim)
   return np.array(train_emb), np.array(test_emb)


def apply_sentence_embeddings(df_train,df_test):
    sentence_vec(df_train)
    sentence_vec(df_test)
   
  


In [21]:
def ngrams_combined(Train_X,Test_X):
    vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,3),binary=True)
    vectorizer1 = CountVectorizer(analyzer='char',ngram_range=(2,5),binary=True)
    cv_xtrain_wg = vectorizer.fit_transform(Train_X)
    cv_xvalid_wg = vectorizer.transform(Test_X)
    cv_xtrain_cg = vectorizer1.fit_transform(Train_X)
    cv_xvalid_cg = vectorizer1.transform(Test_X)
    combined_features_train= hstack([cv_xtrain_wg, cv_xtrain_cg], 'csr')
    combined_features_valid = hstack([cv_xvalid_wg,cv_xvalid_cg],'csr')
    return combined_features_train , combined_features_valid

In [22]:
def evaluate_models_Feature_Eng_BERT(df_train,df_test,Train_Y,Test_Y):
  apply_sentence_embeddings(df_train,df_test)
  train_x =  np.array(df_train['sen_emb'].tolist())
  valid_x = np.array(df_test['sen_emb'].tolist())
  scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
  return scores

def evaluate_models_Feature_Eng_w2v(df_train,df_test,Train_Y,Test_Y):
     train_x , valid_x = apply_embeddings(df_train,df_test,model_w2v,300)
     scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
     return scores


def evaluate_models_Feature_Eng_glove(df_train,df_test,Train_Y,Test_Y):
     train_x , valid_x = apply_embeddings(df_train,df_test,model_glove,200)
     scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y)
     return scores

def evaluate_models_Feature_Eng_ngrams_comb(Train_X,Test_X,Train_Y,Test_Y):
    train_x , valid_x = ngrams_combined(Train_X,Test_X)
    scores = basic_classifier_tester(classifiers,train_x,Train_Y,valid_x,Test_Y) 
    return scores


def basic_classifier_tester(classifiers, X_train,Y_train,X_test,Y_test):
    report_scores = {}
    for classifier in (classifiers):
        classifier.fit(X_train, Y_train)
        predictions = classifier.predict(X_test)
        report = classification_report(Y_test, predictions,output_dict=True)
        report_scores[classifier.__class__.__name__] = report
        # print(classifier.__class__.__name__ + ":")
        # print(report)
        # print('------------------------------------------------------')
    return report_scores


## Evaluation per target and all target 

In [37]:
targets = ['hil','cc','ab','ath','fm','all']

def run_all_models(df_train,df_test,Train_X,Train_Y,Test_X,Test_Y):
    scores_0 = evaluate_models_Feature_Eng_ngrams_comb(Train_X,Test_X,Train_Y,Test_Y)
    scores_1 =evaluate_models_Feature_Eng_w2v(df_train,df_test,Train_Y,Test_Y)
    scores_2 = evaluate_models_Feature_Eng_glove(df_train,df_test,Train_Y,Test_Y)
    scores_3 = evaluate_models_Feature_Eng_BERT(df_train,df_test,Train_Y,Test_Y)
    return scores_0,scores_1,scores_2,scores_3

def put_scores(scores_0,scores_1,scores_2,scores_3):
    scores = {}
    scores['ngram'] = scores_0
    scores['w2v'] = scores_1
    scores['glove'] = scores_2
    scores['BERT'] = scores_3
    return scores

def Evaluation():
   dicts = {}
   print('Evaluation beginning /////////////////////////')
   for i in range(len(targets)):
     if targets[i] == 'hil':
       scores_0, scores_1,scores_2,scores_3 = run_all_models(df_train_hillary,df_test_hillary,Train_X_hillary,Train_Y_hillary,Test_X_hillary,Test_Y_hillary)
       scores  = put_scores(scores_0,scores_1,scores_2,scores_3)
       dicts['Hillary'] = scores
     elif targets[i] == 'cc':
       scores_0,scores_1,scores_2,scores_3 = run_all_models(df_train_climatechange,df_test_climatechange,Train_X_climatechange,Train_Y_climatechange,Test_X_climatechange,Test_Y_climatechange)
       scores  = put_scores(scores_0,scores_1,scores_2,scores_3)
       dicts['Climate change'] = scores
     elif  targets[i] == 'ab':
       scores_0,scores_1,scores_2,scores_3 = run_all_models(df_train_abortion,df_test_abortion,Train_X_abortion,Train_Y_abortion,Test_X_abortion,Test_Y_abortion)
       scores  = put_scores(scores_0,scores_1,scores_2,scores_3)
       dicts['Abortion'] = scores
     elif targets[i] == 'ath':
       scores_0,scores_1,scores_2,scores_3 = run_all_models(df_train_atheism,df_test_atheism,Train_X_atheism,Train_Y_atheism,Test_X_atheism,Test_Y_atheism)
       scores  = put_scores(scores_0,scores_1,scores_2,scores_3)
       dicts['Atheism'] = scores
     elif targets[i] == 'fm':
       scores_0,scores_1,scores_2,scores_3 = run_all_models(df_train_feministmovement,df_test_feministmovement,Train_X_feministmovement,Train_Y_feministmovement,Test_X_feministmovement,Test_Y_feministmovement)
       scores  = put_scores(scores_0,scores_1,scores_2,scores_3)
       dicts['Feminist movement'] = scores
     else:
       scores_0,scores_1,scores_2,scores_3 = run_all_models(df_train,df_test,Train_X,Train_Y,Test_X,Test_Y)
       scores  = put_scores(scores_0,scores_1,scores_2,scores_3)
       dicts['All'] = scores
   print('Evaluation completed ////////////////////////')
   return dicts




In [38]:
evaluation = Evaluation()

Evaluation beginning /////////////////////////


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f62

KeyboardInterrupt: ignored

In [ ]:
# evaluation['All']['ngram']['MLPClassifier']

#Fine Tune SVM  AND MLP

### Linear SVC 

In [27]:
def finetune_linearsvc(Train_X,Train_Y,Test_X,Test_Y):
    C = [2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),1,2,4,8,16,32,100] 
    parameter_space = [ {'C': C}]
    clf_svm_linear = GridSearchCV(LinearSVC(), 
                               parameter_space,  
                               cv =folds,
                               n_jobs=-1,
                               verbose=True    
                              )    

    t0 =time.time()
# fit the model
    clf_svm_linear.fit(Train_X, Train_Y)
    t1= time.time()
    print("training time:", round(t1-t0, 3), "s") # the time would be round to 3 decimal in seconds
    print(clf_svm_linear.best_params_)

    svm = LinearSVC(C = clf_svm_linear.best_params_['C'],max_iter=1500)
    svm.fit(Train_X, Train_Y)
    predictions = svm.predict(Test_X)
    report = classification_report(Test_Y, predictions,output_dict=True)
    return svm
    

### RBF SVC

In [20]:

def finetune_rbf(Train_X,Train_Y,Test_X,Test_Y):
    C = [2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),1,2,4,8,16,32] 
    gammas = np.linspace(10**-5,5,10)
    parameter_space = [ {'C': C, 'gamma': gammas}]
    clf_svm_rbf = GridSearchCV(SVC(kernel='rbf'), 
                               parameter_space,  
                               cv =folds,
                               n_jobs=-1,
                               verbose=True    
                              )      

    t0 =time.time()

# fit the model
    clf_svm_rbf.fit(Train_X, Train_Y)
    t1= time.time()
    print("training time:", round(t1-t0, 3), "s") # the time would be round to 3 decimal in seconds
    print(clf_svm_rbf.best_params_)

    svm = SVC(kernel='rbf',gamma= clf_svm_rbf.best_params_['gamma'],C = clf_svm_rbf.best_params_['C'],max_iter=1500)
    svm.fit(Train_X, Train_Y)
    predictions = svm.predict(Test_X)
    report = classification_report(Test_Y, predictions,output_dict=True)
    return report, svm

### MLP

In [21]:
def finetune_mlp(Train_X,Train_Y,Test_X,Test_Y):
    parameter_space = {
       'hidden_layer_sizes': [(10,30,10),(16,64,16),(64,128,64)],
       'activation': ['sigmoid', 'relu'],
       'solver': ['sgd', 'adam'],
       'alpha': [0.0001, 0.001,0.001,0.05],
       'learning_rate': ['constant','adaptive'] }
    clf_mlp = GridSearchCV(MLPClassifier(max_iter=1500), parameter_space, n_jobs=-1, cv=folds,verbose=True)


    t0 =time.time()

# fit the model
    clf_mlp.fit(Train_X, Train_Y)
    t1= time.time()
    print("training time:", round(t1-t0, 3), "s") # the time would be round to 3 decimal in seconds
    print(clf_mlp.best_params_)
    
    mlp = MLPClassifier(max_iter=1500,hidden_layer_sizes=clf_mlp.best_params_['hidden_layer_sizes'],activation=clf_mlp.best_params_['activation'],
                        solver = clf_mlp.best_params_['solver'],alpha=clf_mlp.best_params_['alpha'],learning_rate =clf_mlp.best_params_['learning_rate'])
    mlp.fit(Train_X,Train_Y)
    predictions = mlp.predict(Test_X)
    report = classification_report(Test_Y,predictions,output_dict=True)
    return report, mlp



## Fine tuned PIPELINE

In [38]:
targets = ['hil','cc','ab','ath','fm','all']



def finetuned_classifier_tester(X_train,Y_train,X_test,Y_test):
    classifier = finetune_linearsvc(X_train,Y_train,X_test,Y_test)
    # report_scores[classifier.__class__.__name__] = report
    # report, classifier = finetune_rbf(X_train,Y_train,X_test,Y_test)
    # report_scores[classifier.__class__.__name__] = report
    # report, classifier = finetune_mlp(X_train,Y_train,X_test,Y_test)
    # report_scores[classifier.__class__.__name__] = report
    return classifier




def finetune_BERT(df_train,df_test,Train_Y,Test_Y):
  apply_sentence_embeddings(df_train,df_test)
  train_x =  np.array(df_train['sen_emb'].tolist())
  valid_x = np.array(df_test['sen_emb'].tolist())
  scores = finetuned_classifier_tester(train_x,Train_Y,valid_x,Test_Y) 
  return scores

def finetune_w2v(df_train,df_test,Train_Y,Test_Y):
     train_x , valid_x = apply_embeddings(df_train,df_test,model_w2v,300)
     scores = finetuned_classifier_tester(train_x,Train_Y,valid_x,Test_Y) 
     return scores


def finetune_glove(df_train,df_test,Train_Y,Test_Y):
     train_x , valid_x = apply_embeddings(df_train,df_test,model_glove,200)
     scores = finetuned_classifier_tester(train_x,Train_Y,valid_x,Test_Y)
     return scores

def finetune_ngrams_comb(Train_X,Test_X,Train_Y,Test_Y):
    train_x , valid_x = ngrams_combined(Train_X,Test_X)
    scores = finetuned_classifier_tester(train_x,Train_Y,valid_x,Test_Y) 
    return scores


def run_all_finetune_models(df_train,df_test,Train_X,Train_Y,Test_X,Test_Y):
    # scores_0 = finetune_ngrams_comb(Train_X,Test_X,Train_Y,Test_Y)
    # scores_1 = finetune_w2v(df_train,df_test,Train_Y,Test_Y)
    # scores_2 = finetune_glove(df_train,df_test,Train_Y,Test_Y)
    scores = finetune_BERT(df_train,df_test,Train_Y,Test_Y)
    return scores



def put_scores(score):
    scores = {}
    # scores['ngram'] = scores_0
    # scores['w2v'] = scores_1
    # scores['glove'] = scores_2
    scores['BERT'] = score
    return scores

def Evaluation_finetune():
   print('Evaluation beginning /////////////////////////')
   for i in range(len(targets)):
     if targets[i] == 'hil':
       svm= run_all_finetune_models(df_train_hillary,df_test_hillary,Train_X_hillary,Train_Y_hillary,Test_X_hillary,Test_Y_hillary)
       ag_tweet_hil = argumentative_tweets.loc[argumentative_tweets['Target'] == 'Hillary Clinton']
       sentence_vec(ag_tweet_hil)
       vector=  np.array(ag_tweet_hil['sen_emb'].tolist())
       predictions = svm.predict(vector)
       ag_tweet_hil['predicted_stance'] = predictions
       mask_against = ag_tweet_hil['predicted_stance'] == 0
       mask_favor = ag_tweet_hil['predicted_stance'] == 1
       model_favor_hil = ag_tweet_hil[mask_favor]
       model_against_hil = ag_tweet_hil[mask_against]
       model_favor_hil.to_pickle('model_favor_hil.pkl')
       model_against_hil.to_pickle('model_against_hil.pkl')
     elif targets[i] == 'cc':
       ag_tweet_cc = argumentative_tweets.loc[argumentative_tweets['Target'] == 'Climate Change is a Real Concern']
       svm=  run_all_finetune_models(df_train_climatechange,df_test_climatechange,Train_X_climatechange,Train_Y_climatechange,Test_X_climatechange,Test_Y_climatechange)
       sentence_vec(ag_tweet_cc)
       vector=  np.array(ag_tweet_cc['sen_emb'].tolist())
       predictions = svm.predict(vector)
       ag_tweet_cc['predicted_stance'] = predictions
       mask_against = ag_tweet_cc['predicted_stance'] == 0
       mask_favor = ag_tweet_cc['predicted_stance'] == 1
       model_favor_cc = ag_tweet_cc[mask_favor]
       model_against_cc = ag_tweet_cc[mask_against]
       model_favor_cc.to_pickle('model_favor_cc.pkl')
       model_against_cc.to_pickle('model_against_cc.pkl')
     elif  targets[i] == 'ab':
       ag_tweet_ab = argumentative_tweets.loc[argumentative_tweets['Target'] ==  'Legalization of Abortion']
       svm = run_all_finetune_models(df_train_abortion,df_test_abortion,Train_X_abortion,Train_Y_abortion,Test_X_abortion,Test_Y_abortion)
       sentence_vec(ag_tweet_ab)
       vector=  np.array(ag_tweet_ab['sen_emb'].tolist())
       predictions = svm.predict(vector)
       ag_tweet_ab['predicted_stance'] = predictions
       mask_against = ag_tweet_ab['predicted_stance'] == 0
       mask_favor = ag_tweet_ab['predicted_stance'] == 1
       model_favor_ab = ag_tweet_ab[mask_favor]
       model_against_ab = ag_tweet_ab[mask_against]
       model_favor_ab.to_pickle('model_favor_ab.pkl')
       model_against_ab.to_pickle('model_against_ab.pkl')
       
     elif targets[i] == 'ath':
       ag_tweet_ath = argumentative_tweets.loc[argumentative_tweets['Target'] ==  'Atheism']
       svm = run_all_finetune_models(df_train_atheism,df_test_atheism,Train_X_atheism,Train_Y_atheism,Test_X_atheism,Test_Y_atheism)
       sentence_vec(ag_tweet_ath)
       vector=  np.array(ag_tweet_ath['sen_emb'].tolist())
       predictions = svm.predict(vector)
       ag_tweet_ath['predicted_stance'] = predictions
       mask_against = ag_tweet_ath['predicted_stance'] == 0
       mask_favor = ag_tweet_ath['predicted_stance'] == 1
       model_favor_ath = ag_tweet_ath[mask_favor]
       model_against_ath = ag_tweet_ath[mask_against]
       model_favor_ath.to_pickle('model_favor_ath.pkl')
       model_against_ath.to_pickle('model_against_ath.pkl')
     elif targets[i] == 'fm':
       svm = run_all_finetune_models(df_train_feministmovement,df_test_feministmovement,Train_X_feministmovement,Train_Y_feministmovement,Test_X_feministmovement,Test_Y_feministmovement)
       ag_tweet_fm = argumentative_tweets.loc[argumentative_tweets['Target'] == 'Feminist Movement']
       sentence_vec(ag_tweet_fm)
       vector=  np.array(ag_tweet_fm['sen_emb'].tolist())
       predictions = svm.predict(vector)
       ag_tweet_fm['predicted_stance'] = predictions
       mask_against = ag_tweet_fm['predicted_stance'] == 0
       mask_favor = ag_tweet_fm['predicted_stance'] == 1
       model_favor_fm = ag_tweet_fm[mask_favor]
       model_against_fm = ag_tweet_fm[mask_against]
       model_favor_fm.to_pickle('model_favor_fm.pkl')
       model_against_fm.to_pickle('model_against_fm.pkl')
     else:
         return 'COMPLETED WOOOOOOO'


In [39]:
evaluation_finetuned = Evaluation_finetune()

Evaluation beginning /////////////////////////


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed:    3.9s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    3.9s finished


training time: 4.506 s
{'C': 0.03125}


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed:    3.7s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    3.8s finished


training time: 5.458 s
{'C': 0.125}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [57]:
 evaluation_finetuned['Feminist movement']['BERT']['LinearSVC']

{'0': {'f1-score': 0.6785714285714286,
  'precision': 0.7450980392156863,
  'recall': 0.6229508196721312,
  'support': 183},
 '1': {'f1-score': 0.41134751773049644,
  'precision': 0.3493975903614458,
  'recall': 0.5,
  'support': 58},
 '2': {'f1-score': 0.4086021505376344,
  'precision': 0.3877551020408163,
  'recall': 0.4318181818181818,
  'support': 44},
 'accuracy': 0.5684210526315789,
 'macro avg': {'f1-score': 0.4995070322798531,
  'precision': 0.4940835772059828,
  'recall': 0.5182563338301044,
  'support': 285},
 'weighted avg': {'f1-score': 0.5825095511599864,
  'precision': 0.6094007926569487,
  'recall': 0.5684210526315789,
  'support': 285}}

In [ ]:
svm = 

In [ ]:
targets = ['hil','cc','ab','ath','fm','all']

In [53]:
with open('evaluation_finetuned.pickle', 'wb') as handle:
    pickle.dump(evaluation_finetuned, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
with open('evaluation.pickle', 'wb') as handle:
    pickle.dump(evaluation, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Error Analysis

In [53]:
def misclassification(clf,df_test,valid_x):
   #index of misclassifications
   y_pred = clf.predict(valid_x)
   correct = y_pred == Test_Y_atheism
   df_test['Classified'] = correct
   classified = df_test[['Tweet','Classified','Target','Stance']]
   return classified

In [54]:
apply_sentence_embeddings(df_train_atheism,df_test_atheism)

In [55]:

train_x =  np.array(df_train_atheism['sen_emb'].tolist())
valid_x = np.array(df_test_atheism['sen_emb'].tolist())
report, clf = finetune_linearsvc(train_x,Train_Y_atheism,valid_x,Test_Y_atheism)



Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed:    4.3s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    4.4s finished


training time: 4.654 s
{'C': 0.125}


In [142]:
# combined_train ,combined_valid =  ngrams_combined(Train_X,Test_X)
train_x , valid_x = apply_embeddings(df_train_hillary,df_test_hillary,model_w2v,300)
report_, clf_ = finetune_linearsvc(train_x,Train_Y_hillary,valid_x,Test_Y_hillary)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.


training time: 0.39 s
{'C': 0.25}


[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.4s finished


In [57]:
classified = misclassification(clf,df_test_atheism,valid_x)
classified_ = misclassification(clf_,df_test_atheism,combined_valid)

In [58]:
misc_ngram  = classified_.loc[classified_['Classified'] == False]
cc_bert = classified.loc[classified['Classified'] == True]

In [59]:
corrected_tweets = []
corrected_stance = []
for index, row in cc_bert.iterrows():
    for index_,row_ in  misc_ngram.iterrows():
         if row['Tweet'] == row_['Tweet']:
               corrected_tweets.append(row['Tweet'])
               corrected_stance.append(row['Stance'])           

In [60]:
corrected_tweets

['Phew! Am glad we came to our senses and worship a 2,000 year old carpenter. - Bart Simpson #jokes #SemST',
 'The #humanist church is the new cloth. #kissTheRing #tcot #SemST',
 'We often talk of unbelief as if it were an affliction to be pitied instead of a crime to be condemned. - Charles Spurgeon #GOD #SemST',
 'Religious holidays should not be public. #RightsAndRecourse #Transformation #SemST',
 'I officiate #humanist & most #nonreligious weddings-starting at $99. Ask me about #student discounts! #216 #celebrant #weddings #SemST',
 'The deeper I go with in- The deeper~> the silence- the peace- the healing- the clarity- the vision- the belief- The deeper my  #SemST',
 '@godless_mom I like how Jesus is reduced to appearing on toast and photoshopped images and they still praise it. #SemST',
 'As opposed to how the media portrays him,the pope is a religious wacko just like any of them and still supports bigotry & misogyny #SemST',
 'We have to #Pray with our eyes on #God, not on the d

In [44]:
cc_ngrams = classified.loc[classified['Classified'] == True] 

In [45]:
cc_ngrams['Tweet']

Int64Index([677, 679, 680, 682, 686, 688, 689, 693, 695, 696,
            ...
            948, 950, 952, 953, 954, 958, 959, 960, 963, 967],
           dtype='int64', length=142)

In [95]:
from sklearn.feature_selection import RFE, chi2, mutual_info_classif,SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [138]:

vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,3),binary=True,)
vectorizer1 = CountVectorizer(analyzer='char',ngram_range=(2,5),binary=True)
cv_xtrain_wg = vectorizer.fit_transform(Train_X)
cv_xvalid_wg = vectorizer.transform(Test_X)
cv_xtrain_cg = vectorizer1.fit_transform(Train_X)
cv_xvalid_cg = vectorizer1.transform(Test_X)
combined_features_train= hstack([cv_xtrain_wg, cv_xtrain_cg], 'csr')
combined_features_valid = hstack([cv_xvalid_wg,cv_xvalid_cg],'csr')
features_1 = vectorizer.get_feature_names()
features_2 = vectorizer1.get_feature_names()
all_features = []
for f in features_1:
  all_features.append(f)

for f in features_2:
  all_features.append(f)



In [ ]:

sel = SelectFromModel(LinearSVC(),max_features=10,prefit=True)
sel.fit(train_x,Train_Y_hillary)

vectors = []
for i in range(len(train_x)):
   vectors.append(train_x[i])
best_features = pd.DataFrame({ 'Rank': sel.get_support(),
                               'vectors': [vectors]})

In [172]:
rank = sel.get_support()

(412, 300)

In [144]:
best = best_features.sort_values(by='Rank',ascending=False)

[('inspirational', 0.2881263792514801)]

(412, 300)

In [145]:
best.head(10)

,Rank
266,0.806079
28,0.641333
91,0.593990
3,0.555043
294,0.542864
130,0.534884
34,0.516292
164,0.515460
163,0.480928
43,0.439281
